# COSC311 Project 2
*JJ McCauley*


## Task 1: Clustering for Wireless Indoor Localization Dataset

In [ ]:
'''Performing clustering on the Wireless Indoor Localization Dataset'''
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import k_means

df = pd.read_csv('wifi_localization.txt')

## Task 2: PCA-based Handwritten Digits Classification 

In [ ]:
''''''